In [ ]:
!pip install bert-for-tf2
!pip install sentencepiece

     |████████████████████████████████| 41 kB 99 kB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.9-py3-none-any.whl size=30536 sha256=2371dae88713785f108281592a6ab475d2de1f07ab40a43c630fab49a40d95e7
  Stored in directory: /root/.cache/pip/wheels/47/b6/e5/8c76ec779f54bc5c2f1b57d2200bb9c77616da83873e8acb53
  Created wheel for params-flow: filename=params_flow-0.8.2-py3-none-any.whl size=19471 sha256=31d5a0d5afbb451e991ca70fbccc33cd356bf30d00bc4adb569df918e94de0fa
  Stored in directory: /root/.cache/pip/wheels/0e/fc/d2/a44fff33af0f233d7def6e7de413006d57c10e10ad736fe8f5
  Created wheel for py-params: filename=py_params-0.10.2-py3-none-any.whl size=7910 sha256=e7879021d5272eae10ef3a38a93e09e4c600419345184c7a729cca62a494b68a
  Stored in directory: /root/.cache/pip/wheels/e1/11/67/33cc51bbee127cb8fb2ba549cd29109b2f22da43ddf9969716
Successfully built bert-for-tf2 params-flow py-params
     |████████████████████████████████| 1.2 MB 16.7 MB/s 


In [ ]:
try:
    %tensorflow_version 2.x
except Exception:
    pass
import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers
import bert

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import math

In [ ]:
df = pd.read_csv('/content/test_9442.tsv', delimiter='\t', header=None)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def data_preprocessing(df):
  df.columns=['Statement', 'Question', 'Answer', 'Label', 'Category']
  df.dropna(inplace=True)
  enc = LabelEncoder()
  df['Label'] = enc.fit_transform(df['Label'])
  enc2 = LabelEncoder()
  df['Category'] = enc2.fit_transform(df['Category'])
  return df

# New Section

In [ ]:

df = data_preprocessing(df)

In [ ]:
df.head()

,Statement,Question,Answer,Label,Category
0,"Durer's father died in 1502, and his mother di...",How long was his mother ill?,she was ill for 30 seconds,0,0
1,"Durer's father died in 1502, and his mother di...",How long was his mother ill?,six centuries,0,0
2,"Durer's father died in 1502, and his mother di...",How long was his mother ill?,she was ill for 90 years,0,0
3,"Durer's father died in 1502, and his mother di...",How long was his mother ill?,6 months,1,0
4,"Durer's father died in 1502, and his mother di...",How long was his mother ill?,six minutes,0,0


In [ ]:
df['Text'] = df['Statement']+df['Question']+df['Answer']

In [ ]:
df = df.drop(['Statement','Question','Answer','Category'], axis=1)

In [ ]:
df.head()

,Label,Text
0,0,"Durer's father died in 1502, and his mother di..."
1,0,"Durer's father died in 1502, and his mother di..."
2,0,"Durer's father died in 1502, and his mother di..."
3,1,"Durer's father died in 1502, and his mother di..."
4,0,"Durer's father died in 1502, and his mother di..."


In [ ]:
BertTokenizer = bert.bert_tokenization.FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

In [ ]:
def tokenize(text):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))

In [ ]:
tokenize= [tokenize(text) for text in df['Text'] ]

In [ ]:
len(tokenize)

9442

In [ ]:
y = df['Label'].values

In [ ]:
tokenize = [(text, y[i]) for i, text in enumerate(tokenize)]

In [ ]:
processed_dataset = tf.data.Dataset.from_generator(lambda: tokenize, output_types=(tf.int32, tf.int32))

In [ ]:
processed_dataset

<FlatMapDataset shapes: (<unknown>, <unknown>), types: (tf.int32, tf.int32)>

In [ ]:
BATCH_SIZE = 32
batched_dataset = processed_dataset.padded_batch(BATCH_SIZE, padded_shapes=((None, ), ()))


In [ ]:
next(iter(batched_dataset))


(<tf.Tensor: shape=(32, 37), dtype=int32, numpy=
 array([[ 4241, 14544,  1005, ...,     0,     0,     0],
        [ 4241, 14544,  1005, ...,     0,     0,     0],
        [ 4241, 14544,  1005, ...,     0,     0,     0],
        ...,
        [ 4241, 14544,  1005, ...,     0,     0,     0],
        [ 4241, 14544,  1005, ...,     0,     0,     0],
        [ 4241, 14544,  1005, ...,     0,     0,     0]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 0, 1, 0], dtype=int32)>)

In [ ]:
next(iter(batched_dataset))


(<tf.Tensor: shape=(32, 37), dtype=int32, numpy=
 array([[ 4241, 14544,  1005, ...,     0,     0,     0],
        [ 4241, 14544,  1005, ...,     0,     0,     0],
        [ 4241, 14544,  1005, ...,     0,     0,     0],
        ...,
        [ 4241, 14544,  1005, ...,     0,     0,     0],
        [ 4241, 14544,  1005, ...,     0,     0,     0],
        [ 4241, 14544,  1005, ...,     0,     0,     0]], dtype=int32)>,
 <tf.Tensor: shape=(32,), dtype=int32, numpy=
 array([0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0,
        0, 0, 0, 1, 1, 0, 0, 0, 1, 0], dtype=int32)>)

In [ ]:
TOTAL_BATCHES = math.ceil(len(tokenize) / BATCH_SIZE)
TEST_BATCHES = TOTAL_BATCHES // 10
batched_dataset.shuffle(TOTAL_BATCHES)
test_data = batched_dataset.take(TEST_BATCHES)
train_data = batched_dataset.skip(TEST_BATCHES)

In [ ]:
class TEXT_MODEL(tf.keras.Model):
    
    def __init__(self,
                 vocabulary_size,
                 embedding_dimensions=128,
                 cnn_filters=50,
                 dnn_units=512,
                 model_output_classes=2,
                 dropout_rate=0.1,
                 training=False,
                 name="text_model"):
        super(TEXT_MODEL, self).__init__(name=name)
        
        self.embedding = layers.Embedding(vocabulary_size,
                                          embedding_dimensions)
        self.cnn_layer1 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=2,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer2 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=3,
                                        padding="valid",
                                        activation="relu")
        self.cnn_layer3 = layers.Conv1D(filters=cnn_filters,
                                        kernel_size=4,
                                        padding="valid",
                                        activation="relu")
        self.pool = layers.GlobalMaxPool1D()
        
        self.dense_1 = layers.Dense(units=dnn_units, activation="relu")
        self.dropout = layers.Dropout(rate=dropout_rate)
        if model_output_classes == 2:
            self.last_dense = layers.Dense(units=1,
                                           activation="sigmoid")
        else:
            self.last_dense = layers.Dense(units=model_output_classes,
                                           activation="softmax")
    
    def call(self, inputs, training):
        l = self.embedding(inputs)
        l_1 = self.cnn_layer1(l) 
        l_1 = self.pool(l_1) 
        l_2 = self.cnn_layer2(l) 
        l_2 = self.pool(l_2)
        l_3 = self.cnn_layer3(l)
        l_3 = self.pool(l_3) 
        
        concatenated = tf.concat([l_1, l_2, l_3], axis=-1) # (batch_size, 3 * cnn_filters)
        concatenated = self.dense_1(concatenated)
        concatenated = self.dropout(concatenated, training)
        model_output = self.last_dense(concatenated)
        
        return model_output

In [ ]:
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200
CNN_FILTERS = 100
DNN_UNITS = 256
OUTPUT_CLASSES = 2

DROPOUT_RATE = 0.2

NB_EPOCHS = 5

In [ ]:
text_model = TEXT_MODEL ( vocabulary_size=VOCAB_LENGTH,
                        embedding_dimensions=EMB_DIM,
                        cnn_filters=CNN_FILTERS,
                        dnn_units=DNN_UNITS,
                        model_output_classes=OUTPUT_CLASSES,
                        dropout_rate=DROPOUT_RATE)

In [ ]:
text_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=[tf.metrics.Precision(), tf.metrics.Recall()])


In [ ]:
text_model.fit(train_data, epochs=NB_EPOCHS)


Epoch 1/5
267/267 [==============================] - 31s 113ms/step - loss: 0.3090 - precision: 0.8205 - recall: 0.7694
Epoch 2/5
267/267 [==============================] - 30s 113ms/step - loss: 0.2711 - precision: 0.8311 - recall: 0.8026
Epoch 3/5
267/267 [==============================] - 30s 112ms/step - loss: 0.2738 - precision: 0.8261 - recall: 0.8064
Epoch 4/5
267/267 [==============================] - 30s 112ms/step - loss: 0.2389 - precision: 0.8586 - recall: 0.8230
Epoch 5/5
267/267 [==============================] - 30s 112ms/step - loss: 0.2000 - precision: 0.8763 - recall: 0.8448


In [ ]:
results = text_model.evaluate(test_data)


29/29 [==============================] - 1s 24ms/step - loss: 0.7140 - precision: 0.5388 - recall: 0.7738


In [ ]:
print("F1 Score : ", 2*((results[1]*results[2])/(results[1]+results[2])))


F1 Score :  0.635262435997939
